# Trash Object Detection Using YOLOv8

This notebook is comparised of several components to deploy an trash detection model using YOLOv5 and YOLOv8. The notebook includes the following: (1) Installation and Setup of YOLOv5 and YOLOv8, (2) Downloading the TACO Dataset, (3) Preprocessing the Data, (4) Model Training with YOLOv5 and YOLOv8, (5) YOLO Model Evaluation, and (6) Trash Detection Model Deployment.
<br>
<br>
Objective(s): Build and train a Custom YOLOv8 Model on the TACO dataset to recognize various types of trash. Create custom Python scripts to deploy the YOLOv8 model on the Freenove Tank Robot that: (a) implement real-time trash detection using the robot's camera and the trained model and (b) program the robot to move towards and pick-up trash if detected or keep searching for trash if no trash was detected. 


## Installation and Preparation

### YOLOv5/YOLOv8
For information on installation and preperation, see https://github.com/mattiegisselbeck/trash-detection.

### TACO

### Run YOLOv5

### Run YOLOv8


| Model                                                                                | size<br><sup>(pixels) | mAP<sup>val<br>50-95 | Speed<br><sup>CPU ONNX<br>(ms) | Speed<br><sup>A100 TensorRT<br>(ms) | params<br><sup>(M) | FLOPs<br><sup>(B) |
| ------------------------------------------------------------------------------------ | --------------------- | -------------------- | ------------------------------ | ----------------------------------- | ------------------ | ----------------- |
| [YOLOv8x](https://github.com/ultralytics/assets/releases/download/v0.0.0/yolov8x.pt) | 640                   | 53.9                 | 479.1                          | 3.53                                | 68.2               | 257.8             |


| Model                                                                                | size<br><sup>(pixels) | mAP<sup>val<br>50-95 | Speed<br><sup>CPU ONNX<br>(ms) | Speed<br><sup>A100 TensorRT<br>(ms) | params<br><sup>(M) | FLOPs<br><sup>(B) |
| ------------------------------------------------------------------------------------ | --------------------- | -------------------- | ------------------------------ | ----------------------------------- | ------------------ | ----------------- |
| [YOLOv8n](https://github.com/ultralytics/assets/releases/download/v0.0.0/yolov8n.pt) | 640                   | 37.3                 | 80.4                           | 0.99                                | 3.2                | 8.7               |
| [YOLOv8s](https://github.com/ultralytics/assets/releases/download/v0.0.0/yolov8s.pt) | 640                   | 44.9                 | 128.4                          | 1.20                                | 11.2               | 28.6              |
| [YOLOv8m](https://github.com/ultralytics/assets/releases/download/v0.0.0/yolov8m.pt) | 640                   | 50.2                 | 234.7                          | 1.83                                | 25.9               | 78.9              |
| [YOLOv8x](https://github.com/ultralytics/assets/releases/download/v0.0.0/yolov8x.pt) | 640                   | 53.9                 | 479.1                          | 3.53                                | 68.2               | 257.8             |


In [ ]:
from ultralytics import YOLO

# Build Model
model = YOLO("/Users/mattiegisselbeck/Documents/GitHub/trashbot/runs/detect/train4/weights/best.pt") 

# Train YOLOv8
results = model.train(data="./data.yaml", epochs=1)

In [ ]:
model = YOLO("/Users/mattiegisselbeck/Documents/GitHub/trashbot/runs/detect/train5/weights/best.pt") 

results = model("/Users/mattiegisselbeck/Desktop/000013.JPG")  # Predict (Test)

### YOLOv8 on Freenove Robot / Raspberry Pi

In [ ]:
import os
import time
import pigpio
from camera import Camera
from motor import Motor
from infrared import Infrared

class TrashBot:
    def __init__(self):
        # Initialize pigpio
        os.system("sudo pigpiod")
        time.sleep(1)  # Wait for pigpiod to initialize

        # Initialize components
        self.camera = Camera()
        self.motor = Motor()
        self.infrared = Infrared()

    def detect_trash(self):
        while True:
            trash_detected, distance = self.camera.look_for_trash()

            if not trash_detected:
                self.motor.move_forward(6)  # Move forward 6 inches
            else:
                self.motor.move_distance(distance)  # Move towards the trash
                self.motor.pick_up_trash()
                self.motor.drop_trash()

                # After handling the trash, loop back to start

In [ ]:
#!/usr/bin/python3
# Capture a JPEG while still running in the preview mode. When you
# capture to a file, the return value is the metadata for that image.
import time
from picamera2 import Picamera2, Preview
from libcamera import Transform
picam2 = Picamera2()
preview_config = picam2.create_preview_configuration(main={"size": (640, 480)},transform=Transform(hflip=1,vflip=1))
picam2.configure(preview_config)
picam2.start_preview(Preview.QTGL)
picam2.start()
time.sleep(2)
metadata = picam2.capture_file("image.jpg")
print(metadata)
picam2.close()

In [ ]:
def find_trash():
    picam2 = Picamera2()
    preview_config = picam2.create_preview_configuration(main={"size": (640, 480)},transform=Transform(hflip=1,vflip=1))
    picam2.configure(preview_config)
    picam2.start_preview(Preview.QTGL)
    picam2.start()
    time.sleep(2)
    metadata = picam2.capture_file("image.jpg")
    print(metadata)
    picam2.close()
    
    # Detect 
    model = YOLO("/Users/mattiegisselbeck/Documents/GitHub/trashbot/runs/detect/train4/weights/best.pt") 
    results = model("/path/to/image.jpg")

In [ ]:
class Ultrasonic:
    def __init__(self):        
        GPIO.setwarnings(False)        
        self.trigger_pin = 27
        self.echo_pin = 22
        self.MAX_DISTANCE = 300               # define the maximum measuring distance, unit: cm
        self.timeOut = self.MAX_DISTANCE*60   # calculate timeout according to the maximum measuring distance
        GPIO.setmode(GPIO.BCM)
        GPIO.setup(self.trigger_pin,GPIO.OUT)
        GPIO.setup(self.echo_pin,GPIO.IN)

    def detect_trash(self, image_path):
        # Use the model to detect objects in the image
        results = model(image_path)

        # Check if any detected object is labeled as 'trash'
        for detection in results.xyxy[0]:  # results.xyxy[0] contains the bounding box coordinates and class
            if detection[-1] == self.trash_class_id:  # Assuming you know the class ID for trash
                return True  # Trash is detected
        return False  # No trash detected

    def run_motor(self, distance, towards_trash=False):
        # ... (existing logic with an added condition for moving towards trash)

    def run(self):
        self.PWM=Motor()
        while True:
            image_path = '/path/to/image.jpg'  # Update with the actual image path or method to capture image
            if self.detect_trash(image_path):
                # If trash is detected, move a certain distance towards it
                # You might want to calculate this distance based on the size or position of the trash in the image
                self.run_motor(None, towards_trash=True)
            else:
                # If no trash is detected, continue normal operation
                distance = self.get_distance()
                self.run_motor(distance)


In [ ]:
import RPi.GPIO as GPIO
import time
from Motor import *
from servo import *

class Ultrasonic:
    def __init__(self):        
        GPIO.setwarnings(False)        
        self.trigger_pin = 27
        self.echo_pin = 22
        self.MAX_DISTANCE = 300               # define the maximum measuring distance, unit: cm
        self.timeOut = self.MAX_DISTANCE*60   # calculate timeout according to the maximum measuring distance
        GPIO.setmode(GPIO.BCM)
        GPIO.setup(self.trigger_pin,GPIO.OUT)
        GPIO.setup(self.echo_pin,GPIO.IN)
        
    def pulseIn(self,pin,level,timeOut): # obtain pulse time of a pin under timeOut
        t0 = time.time()
        while(GPIO.input(pin) != level):
            if((time.time() - t0) > timeOut*0.000001):
                return 0;
        t0 = time.time()
        while(GPIO.input(pin) == level):
            if((time.time() - t0) > timeOut*0.000001):
                return 0;
        pulseTime = (time.time() - t0)*1000000
        return pulseTime
    
    def get_distance(self):     # get the measurement results of ultrasonic module,with unit: cm
        distance_cm2=[0.0,0.0,0.0,0.0,0.0]
        for i in range(5):
            GPIO.output(self.trigger_pin,GPIO.HIGH)      # make trigger_pin output 10us HIGH level 
            time.sleep(0.00001)     # 10us
            GPIO.output(self.trigger_pin,GPIO.LOW) # make trigger_pin output LOW level 
            pingTime = self.pulseIn(self.echo_pin,GPIO.HIGH,self.timeOut)   # read plus time of echo_pin
            distance_cm2[i] = pingTime * 340.0 / 2.0 / 10000.0     # calculate distance with sound speed 340m/s
        distance_cm2=sorted(distance_cm2)
        return  distance_cm2[2]
    
    def run_motor(self,distance):
        if(distance!=0):
            if distance < 45 :
                self.PWM.setMotorModel(-1500,-1500) #Back
                time.sleep(0.4)
                self.PWM.setMotorModel(-1500,1500)  #Left
                time.sleep(0.2)         
            else :
                self.PWM.setMotorModel(1500,1500)   #Forward
            
    def run(self):
        self.PWM=Motor()
        while True:
            distance = self.get_distance()
            time.sleep(0.2)
            #print ("The distance is "+str(distance)+"CM")
            self.run_motor(distance)

ultrasonic=Ultrasonic()         
if __name__ == '__main__':     # Program entrance
    print ('Program is starting...')
    servo=Servo()
    servo.setServoPwm('0',90)
    servo.setServoPwm('1',140)
    try:
        ultrasonic.run()
    except KeyboardInterrupt:  # When 'Ctrl+C' is pressed, the child program destroy() will be  executed.
        PWM.setMotorModel(0,0)
        servo.setServoPwm('0',90)
        servo.setServoPwm('1',140)
        print ("\nEnd of program")

import time
from picamera2 import Picamera2, Preview
from libcamera import Transform

def find_trash
    picam2 = Picamera2()
    preview_config = picam2.create_preview_configuration(main={"size": (640, 480)},transform=Transform(hflip=1,vflip=1))
    picam2.configure(preview_config)
    picam2.start_preview(Preview.QTGL)
    picam2.start()
    time.sleep(2)
    metadata = picam2.capture_file("image.jpg")
    print(metadata)
    picam2.close()
    
    # Detect 
    model = YOLO("/Users/mattiegisselbeck/Documents/GitHub/trashbot/runs/detect/train4/weights/best.pt") 
    results = model(/path/to/image.jpg")

    if trash is detected



    
use camera to look for trash (YOLOv8)
        if no trash is detected 
            then move forward 6 inches 
        if trash is detected 
            then calculate the distance to move towards the trash 
            and then move that distance
            and then pick-up trash and move it
            once object is picked-up > drop it

            loop back to the start


This notebook represents a MVP of Trashbot since it holds all the necessary components to deploy, train, and test an trash detection model using YOLOv5 or YOLOv8 and the TACO dataset. A majority of the notebook is written in CLI to allow for simplicity and efficency and that we are able to create a custom conda environment for Trashbot. The notebook also covers CLI instructions on how to download the TACO dataset, download its requirements, and split the dataset for training for the YOLO models. Additionally, the notebook clones the YOLOv5 Github repository, installs ultralytics (necessary for YOLO), builds a new YOLO model, and trains a YOLO model. Near the end of the notebook, there is a draft version of the script that will be used for deploying the YOLOv8 model onto the Freenove robot (Trashbot).